In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.6 MB/s eta 0:00:00


In [8]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
import string

In [4]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/Elsevier /Pranita Project /df_clean (1).csv') ##/content/df_clean.csv
data.head()

,Unnamed: 0,Table_title,concept_1,predicate,concept_2
0,0,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,has clinical finding,mild elevation ALT/AST level
1,1,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,has clinical finding,moderate elevation GGT level
2,2,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,has clinical finding,marked elevation ALP level
3,3,Hepatitic and cholestatic/obstructive liver fu...,hepatitis,has clinical finding,marked elevation ALT/AST level
4,4,Hepatitic and cholestatic/obstructive liver fu...,hepatitis,has clinical finding,mild elevation GGT level


In [5]:
data.shape

(1686, 5)

In [6]:
import string

def clean_string(text):
    # Split text into words
    words = text.split()
    clean_words = []
    for word in words:
        # Remove punctuation and digits from word
        word = word.translate(str.maketrans('', '', string.punctuation + string.digits))
        # Only keep words with more than one character
        if len(word) > 1:
            clean_words.append(word)
    # Join cleaned words into a single string
    clean_text = ' '.join(clean_words)
    return clean_text

In [7]:
data = data.fillna('')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder

In [10]:
from sklearn.preprocessing import LabelEncoder
# Create a label encoder
encoder = LabelEncoder()

# Fit the encoder on the predicate column
encoder.fit(data['predicate'])

# Transform the predicate column
predicate = encoder.transform(data['predicate'])


In [11]:
data['predicate'].value_counts()

has cause                                   681
has clinical finding                        427
has indication                              170
has positive lab value                      159
has topological description/distribution     67
has risk factor                              42
Has clinical finding                         31
Has drug                                     19
Has indication                               19
Has time                                     16
Has function                                 15
has abnormality                              11
Has morphology                                8
is risk factor                                8
Has time|clinical finding                     5
Has quality                                   4
has diagnosis                                 3
has cause                                     1
Name: predicate, dtype: int64

In [12]:
import torch
print(len((predicate)))
# predicate = encoder.transform(data['predicate'])

labels = torch.tensor(predicate)
print(len(set(predicate)))

1686
18


In [13]:
len(labels)

1686

In [14]:
numpy_array = labels.numpy()

# Convert numpy array to a dataframe
df_labels = pd.DataFrame(numpy_array,columns = ['predicate'])

# Print the dataframe
print(df_labels)

      predicate
0            11
1            11
2            11
3            11
4            11
...         ...
1681          9
1682         14
1683         14
1684         14
1685         14

[1686 rows x 1 columns]


In [15]:
df_x = data.drop(columns = ['predicate'])

In [16]:
combined_df = pd.concat([df_x, df_labels], axis = 1)

In [17]:
combined_df

,Unnamed: 0,Table_title,concept_1,concept_2,predicate
0,0,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,mild elevation ALT/AST level,11
1,1,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,moderate elevation GGT level,11
2,2,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,marked elevation ALP level,11
3,3,Hepatitic and cholestatic/obstructive liver fu...,hepatitis,marked elevation ALT/AST level,11
4,4,Hepatitic and cholestatic/obstructive liver fu...,hepatitis,mild elevation GGT level,11
...,...,...,...,...,...
1681,1723,Disseminated intravascular coagulation (DIC),Disseminated intravascular coagulation (DIC),"Toxic/immunological (ex. ABO incompatibility, ...",9
1682,1724,Disseminated intravascular coagulation (DIC),Disseminated intravascular coagulation (DIC),Platelets < 100,14
1683,1725,Disseminated intravascular coagulation (DIC),Disseminated intravascular coagulation (DIC),Elevated fibrin degradation,14
1684,1726,Disseminated intravascular coagulation (DIC),Disseminated intravascular coagulation (DIC),Prolonged prothrombin time,14


In [18]:
combined_df['predicate'].value_counts()

9     681
11    427
13    170
14    159
16     67
15     42
0      31
1      19
3      19
6      16
2      15
8      11
4       8
17      8
7       5
5       4
12      3
10      1
Name: predicate, dtype: int64

In [19]:
#class 10

import pandas as pd

# Select the label you want to duplicate
label_to_duplicate = 10

# Filter the dataframe to select rows with the label
rows_to_duplicate = combined_df[combined_df['predicate'] == label_to_duplicate]

# Duplicate the selected rows using repeat()
duplicated_rows = pd.concat([rows_to_duplicate] * 6, ignore_index=True)

# Concatenate the duplicated rows with the original dataframe
updated_combined_df = pd.concat([combined_df, duplicated_rows], ignore_index=True)

# Print the updated combined dataframe with duplicated rows
print("\nUpdated Combined DataFrame:")
print(updated_combined_df)


Updated Combined DataFrame:
      Unnamed: 0                                        Table_title  \
0              0  Hepatitic and cholestatic/obstructive liver fu...   
1              1  Hepatitic and cholestatic/obstructive liver fu...   
2              2  Hepatitic and cholestatic/obstructive liver fu...   
3              3  Hepatitic and cholestatic/obstructive liver fu...   
4              4  Hepatitic and cholestatic/obstructive liver fu...   
...          ...                                                ...   
1687        1023  US Centers for Disease Control (CDC) definitio...   
1688        1023  US Centers for Disease Control (CDC) definitio...   
1689        1023  US Centers for Disease Control (CDC) definitio...   
1690        1023  US Centers for Disease Control (CDC) definitio...   
1691        1023  US Centers for Disease Control (CDC) definitio...   

                concept_1                       concept_2  predicate  
0     biliary obstruction    mild elevation ALT

In [20]:
#class 12

import pandas as pd

# Select the label you want to duplicate
label_to_duplicate = 12

# Filter the dataframe to select rows with the label
rows_to_duplicate = updated_combined_df[updated_combined_df['predicate'] == label_to_duplicate]

# Duplicate the selected rows using repeat()
duplicated_rows = pd.concat([rows_to_duplicate] * 2, ignore_index=True)

# Concatenate the duplicated rows with the original dataframe
updated_2_df = pd.concat([updated_combined_df, duplicated_rows], ignore_index=True)

# Print the updated combined dataframe with duplicated rows
print("\nUpdated Combined DataFrame:")
print(updated_2_df)


Updated Combined DataFrame:
      Unnamed: 0                                        Table_title  \
0              0  Hepatitic and cholestatic/obstructive liver fu...   
1              1  Hepatitic and cholestatic/obstructive liver fu...   
2              2  Hepatitic and cholestatic/obstructive liver fu...   
3              3  Hepatitic and cholestatic/obstructive liver fu...   
4              4  Hepatitic and cholestatic/obstructive liver fu...   
...          ...                                                ...   
1693         993  Clinical features and staging of tuberculous m...   
1694         994  Clinical features and staging of tuberculous m...   
1695         992  Clinical features and staging of tuberculous m...   
1696         993  Clinical features and staging of tuberculous m...   
1697         994  Clinical features and staging of tuberculous m...   

                                              concept_1  \
0                                   biliary obstruction   


In [21]:
#class 5

import pandas as pd

# Select the label you want to duplicate
label_to_duplicate = 5

# Filter the dataframe to select rows with the label
rows_to_duplicate = updated_2_df[updated_2_df['predicate'] == label_to_duplicate]

# Duplicate the selected rows using repeat()
duplicated_rows = pd.concat([rows_to_duplicate] * 2, ignore_index=True)

# Concatenate the duplicated rows with the original dataframe
updated_3_df = pd.concat([updated_2_df, duplicated_rows], ignore_index=True)

# Print the updated combined dataframe with duplicated rows
print("\nUpdated Combined DataFrame:")
print(updated_3_df)


Updated Combined DataFrame:
      Unnamed: 0                                        Table_title  \
0              0  Hepatitic and cholestatic/obstructive liver fu...   
1              1  Hepatitic and cholestatic/obstructive liver fu...   
2              2  Hepatitic and cholestatic/obstructive liver fu...   
3              3  Hepatitic and cholestatic/obstructive liver fu...   
4              4  Hepatitic and cholestatic/obstructive liver fu...   
...          ...                                                ...   
1701         394  Clinical and investigation findings in the imm...   
1702         391  Clinical and investigation findings in the imm...   
1703         392  Clinical and investigation findings in the imm...   
1704         393  Clinical and investigation findings in the imm...   
1705         394  Clinical and investigation findings in the imm...   

                concept_1                       concept_2  predicate  
0     biliary obstruction    mild elevation ALT

In [22]:
#class 7

import pandas as pd

# Select the label you want to duplicate
label_to_duplicate = 7

# Filter the dataframe to select rows with the label
rows_to_duplicate = updated_3_df[updated_3_df['predicate'] == label_to_duplicate]

# Duplicate the selected rows using repeat()
duplicated_rows = pd.concat([rows_to_duplicate] * 2, ignore_index=True)

# Concatenate the duplicated rows with the original dataframe
updated_4_df = pd.concat([updated_3_df, duplicated_rows], ignore_index=True)

# Print the updated combined dataframe with duplicated rows
print("\nUpdated Combined DataFrame:")
print(updated_4_df)


Updated Combined DataFrame:
      Unnamed: 0                                        Table_title  \
0              0  Hepatitic and cholestatic/obstructive liver fu...   
1              1  Hepatitic and cholestatic/obstructive liver fu...   
2              2  Hepatitic and cholestatic/obstructive liver fu...   
3              3  Hepatitic and cholestatic/obstructive liver fu...   
4              4  Hepatitic and cholestatic/obstructive liver fu...   
...          ...                                                ...   
1711          84                    Causes of pruritus in pregnancy   
1712          87                    Causes of pruritus in pregnancy   
1713          90                    Causes of pruritus in pregnancy   
1714          93                    Causes of pruritus in pregnancy   
1715          96                    Causes of pruritus in pregnancy   

                                              concept_1  \
0                                   biliary obstruction   


In [23]:
updated_4_df.shape

(1716, 5)

In [24]:
updated_4_df.head()

,Unnamed: 0,Table_title,concept_1,concept_2,predicate
0,0,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,mild elevation ALT/AST level,11
1,1,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,moderate elevation GGT level,11
2,2,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,marked elevation ALP level,11
3,3,Hepatitic and cholestatic/obstructive liver fu...,hepatitis,marked elevation ALT/AST level,11
4,4,Hepatitic and cholestatic/obstructive liver fu...,hepatitis,mild elevation GGT level,11


In [31]:
predicate = updated_4_df['predicate'].values
predicate

array([11, 11, 11, ...,  7,  7,  7])

In [32]:
##rough

categorical_features = ['Table_title', 'concept_1', 'concept_2'] #not taking predicate as that has already been converted to integer by label encoder
ordinal_encoder = OrdinalEncoder()
data_encoded = ordinal_encoder.fit_transform(updated_4_df[categorical_features])

smote = SMOTE(random_state=42)
data_resampled, predicate_resampled = smote.fit_resample(data_encoded, predicate)

In [43]:
print(data_resampled.shape)
print(predicate_resampled.shape)

(12258, 3)
(12258,)


In [44]:
data_resampled

array([[  60.        ,  544.        , 1196.        ],
       [  60.        ,  544.        , 1198.        ],
       [  60.        ,  544.        , 1193.        ],
       ...,
       [ 101.        ,  439.        ,  715.60813628],
       [ 101.        ,  439.        , 1058.47877259],
       [ 101.        ,  439.        ,  679.49545133]])

In [40]:
#Unnamed column is creating interference so while reconstructing the new dataframe so we are dropping that column

data = data.drop('Unnamed: 0', axis=1)
data.head()

,Table_title,concept_1,predicate,concept_2
0,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,has clinical finding,mild elevation ALT/AST level
1,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,has clinical finding,moderate elevation GGT level
2,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,has clinical finding,marked elevation ALP level
3,Hepatitic and cholestatic/obstructive liver fu...,hepatitis,has clinical finding,marked elevation ALT/AST level
4,Hepatitic and cholestatic/obstructive liver fu...,hepatitis,has clinical finding,mild elevation GGT level


In [48]:
import numpy as np

data1 = np.column_stack((data_resampled, predicate_resampled))
columns = ['Table_title', 'concept_1', 'predicate', 'concept_2']
new_df = pd.DataFrame(data1, columns=columns, dtype = int)

<ipython-input-48-a74e4f8dcc69>:5: FutureWarning: In a future version, passing float-dtype values and an integer dtype to DataFrame will retain floating dtype if they cannot be cast losslessly (matching Series behavior). To retain the old behavior, use DataFrame(data).astype(dtype)
  new_df = pd.DataFrame(data1, columns=columns, dtype = int)


In [50]:
new_df.head()
data_resampled = new_df
data_resampled

,Table_title,concept_1,predicate,concept_2
0,60,544,1196,11
1,60,544,1198,11
2,60,544,1193,11
3,60,546,1194,11
4,60,546,1197,11
...,...,...,...,...
12253,101,439,735,17
12254,101,439,923,17
12255,101,439,715,17
12256,101,439,1058,17


In [53]:
## rough
num_classes = 18
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=num_classes)


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

In [54]:
text = data_resampled['Table_title'].astype(str) + ' ' + data_resampled['concept_1'].astype(str) + ' ' + data_resampled['concept_2'].astype(str)
inputs = tokenizer(text.tolist(), return_tensors="pt", padding=True, truncation=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [55]:
labels = torch.tensor(predicate_resampled)
labels

tensor([11, 11, 11,  ..., 17, 17, 17])

In [56]:
##rough
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size)

learning_rate = 1e-5
gradient_accumulation_steps = 4

optimizer = Adam(model.parameters(), lr=learning_rate)

model.train()
for epoch in range(3):
    for i, (input_ids, attention_mask, labels) in enumerate(dataloader):
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps
        loss.backward()

        if (i + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

In [59]:
print(inputs['input_ids'].shape) # Should be (batch_size, sequence_length)

# Check the shape of the target tensor
print(labels.shape) # Should be (batch_size,)

torch.Size([1, 14])
torch.Size([2])


In [61]:
data_resampled['predicate'][0]

1196

In [62]:
data['predicate'][0]

'has clinical finding'

In [57]:
# Put the model in evaluation mode
model.eval()

# Prepare your input data
text = "Hepatitic and cholestaticobstructive liver function tests biliary obstruction mild elevation ALTAST level"
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

# Get the predictions
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    predictions = outputs.logits.argmax(dim=-1)
# Print the prediction
print(f"The predicted class is: {predictions.item()}")

The predicted class is: 17


In [58]:
# Put the model in evaluation mode
model.eval()

# Prepare your input data
text = "Megaloblastic anemia Elevated ferritin level"
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

# Get the predictions
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    predictions = outputs.logits.argmax(dim=-1)

# Get the predicted class label
predicted_class_label = encoder.inverse_transform([predictions.item()])[0]

# Print the predicted class label
print(f"The predicted class is: {predicted_class_label}")

The predicted class is: is risk factor


In [64]:
model.save_pretrained("finetuned bioclinicalbert")

In [ ]:
###done here

In [ ]:
###below is different

In [ ]:
# Set the number of classes
num_classes = 18 # Set this to the number of classes in your data
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=num_classes)
# model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

In [ ]:
#now start prepping the data and apply SMOTE

In [ ]:
updated_4_df['Table_title'] = updated_4_df['Table_title'].astype(str).apply(clean_string)
updated_4_df['concept_1'] = updated_4_df['concept_1'].astype(str).apply(clean_string)
updated_4_df['concept_2'] = updated_4_df['concept_2'].astype(str).apply(clean_string)

In [ ]:
updated_4_df.head()

,Unnamed: 0,Table_title,concept_1,concept_2,predicate
0,0,Hepatitic and cholestaticobstructive liver fun...,biliary obstruction,mild elevation ALTAST level,11
1,1,Hepatitic and cholestaticobstructive liver fun...,biliary obstruction,moderate elevation GGT level,11
2,2,Hepatitic and cholestaticobstructive liver fun...,biliary obstruction,marked elevation ALP level,11
3,3,Hepatitic and cholestaticobstructive liver fun...,hepatitis,marked elevation ALTAST level,11
4,4,Hepatitic and cholestaticobstructive liver fun...,hepatitis,mild elevation GGT level,11


In [ ]:
# Tokenize the text
text = updated_4_df['Table_title'] + ' ' + updated_4_df['concept_1'] + ' ' + updated_4_df['concept_2']
inputs = tokenizer(text.tolist(), return_tensors="pt", padding=True, truncation=True)

# Convert the inputs to a PyTorch tensor
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
attention_mask.shape

torch.Size([1716, 107])

In [ ]:
#Now applying SMOTE on input_ids as they contain the encoded tokens

In [ ]:
#applying SMOTE on attention mask as well

In [ ]:
# Data preparation rough
# Prepare the data

# Convert the columns to strings
data['Table_title'] = data['Table_title'].astype(str).apply(clean_string)
data['concept_1'] = data['concept_1'].astype(str).apply(clean_string)
data['concept_2'] = data['concept_2'].astype(str).apply(clean_string)

# Tokenize the text
text = data['Table_title'] + ' ' + data['concept_1'] + ' ' + data['concept_2']
inputs = tokenizer(text.tolist(), return_tensors="pt", padding=True, truncation=True)
# Create the labels tensor
labels = torch.tensor(predicate)
# Convert the inputs to a PyTorch tensor
input_ids = inputs['input_ids']                        ###########doubt#########
attention_mask = inputs['attention_mask']              ###########doubt#########

# Create a DataLoader
batch_size = 8 # Reduce the batch size
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size)

# Set the learning rate and gradient accumulation steps
learning_rate = 1e-5
gradient_accumulation_steps = 4

# Create the optimizer
optimizer = Adam(model.parameters(), lr=learning_rate)

# Fine-tune the model
model.train()
for epoch in range(3):
    for i, (input_ids, attention_mask, labels) in enumerate(dataloader):
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps
        loss.backward()

        if (i + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyboardInterrupt: ignored